In [1]:
import os
import json
from dotenv import load_dotenv
from LLM_API import AuthenticatedRequestSender
from LLM_API import OpenAIService
from FileProcess import FileProcessor
from Memory import Embedding, LocalMemory
from Chat import JoiningChat
from WebExecuter import WebSearch
from FileProcess import CodeAnalyser
from Tools import FunctionManager

analyser=CodeAnalyser()

dotenv_path = os.path.join(os.getcwd(), '.env')
root_path = dotenv_path.rstrip('.env')
load_dotenv(dotenv_path)

fileprocessor=FileProcessor(root_path)

project_root = root_path
function_manager = FunctionManager(project_root, service_type='sensetime')

{'functions': [{'name': 'sum',
   'description': "The function 'sum' takes two arguments, 'a' and 'b', and returns their sum. It is designed to perform basic arithmetic operations and can be used in a variety of contexts where numerical values need to be added together.",
   'input': ['a', 'b'],
   'output': "BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))",
   'file_path': 'Tools\\tools.py',
   'dependencies': []},
  {'name': 'prime_fib_sum',
   'description': '这个函数的功能是计算前n个素数之和与前n个斐波那契数之和，并返回两者的总和。首先，它定义了一个辅助函数is_prime来判断一个数是否为素数。然后，通过循环遍历，累加前n个素数的值，同时也累加前n个斐波那契数的值。最后，返回两者的总和。如果输入的n小于等于0，则返回0作为结果。',
   'input': ['n'],
   'output': "BinOp(left=Name(id='sum_primes', ctx=Load()), op=Add(), right=Name(id='sum_fibs', ctx=Load()))",
   'file_path': 'Tools\\tools.py',
   'dependencies': []}]}

In [1]:
pip install prettytable


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
from prettytable import PrettyTable


In [2]:
import re

question = "What is the sum of 3+5?"
print(function_manager.functions_info)
ask = f"Based on the following function descriptions: '{function_manager.functions_info}', could you demonstrate how to address '{question}'? Please include a specific code snippet that performs this operation. It is highly recommended to enclose your code within triple backticks and specify 'python' as the language, like so: ```python\n# Your code here\n```. This formatting helps ensure that your code can be correctly identified and processed."
print(ask)

{'functions': [{'name': 'sum', 'description': "The function 'sum' takes two arguments, 'a' and 'b', and returns their sum. It is designed to perform basic arithmetic operations and can be used in a variety of contexts where numerical values need to be added together.", 'input': ['a', 'b'], 'output': "BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))", 'file_path': 'Tools\\tools.py', 'dependencies': []}, {'name': 'prime_fib_sum', 'description': '这个函数的功能是计算前n个素数之和与前n个斐波那契数之和，并返回两者的总和。首先，它定义了一个辅助函数is_prime来判断一个数是否为素数。然后，通过循环遍历，累加前n个素数的值，同时也累加前n个斐波那契数的值。最后，返回两者的总和。如果输入的n小于等于0，则返回0作为结果。', 'input': ['n'], 'output': "BinOp(left=Name(id='sum_primes', ctx=Load()), op=Add(), right=Name(id='sum_fibs', ctx=Load()))", 'file_path': 'Tools\\tools.py', 'dependencies': []}]}
Based on the following function descriptions: '{'functions': [{'name': 'sum', 'description': "The function 'sum' takes two arguments, 'a' and 'b', and returns their sum. It is designed to perform basic 

In [5]:
import re
import importlib
import os
import json
from dotenv import load_dotenv
from LLM_API import AuthenticatedRequestSender
from LLM_API import OpenAIService
from FileProcess import FileProcessor
from Memory import Embedding, LocalMemory
from Chat import JoiningChat
from WebExecuter import WebSearch
from FileProcess import CodeAnalyser
from Tools import FunctionManager

analyser=CodeAnalyser()

dotenv_path = os.path.join(os.getcwd(), '.env')
root_path = dotenv_path.rstrip('.env')
load_dotenv(dotenv_path)

fileprocessor=FileProcessor(root_path)

project_root = root_path
function_manager = FunctionManager(project_root, service_type='sensetime')

service_type = 'sensetime'
chat = JoiningChat(service_type=service_type, root_path=root_path)
question = "What is the sum of 3+5?"
ask = f"Based on the following function descriptions: '{function_manager.functions_info}', could you demonstrate how to address '{question}'? Please include a specific code snippet that performs this operation, assigns the result to a variable named 'result', and also assigns the name of the used function to a variable named 'used_function_name'. It is highly recommended to enclose your code within triple backticks and specify 'python' as the language, like so: ```python\n# Your code here\nresult = <operation>\nused_function_name = '<function_name>'\n```. This formatting helps ensure that your code can be correctly identified and processed."
answer=chat.process_input(ask)
# 使用正则表达式提取代码段
pattern = r"```python\n(.*?)\n```"
match = re.search(pattern, answer, re.DOTALL)

if match:
    original_code = match.group(1)
    code_string = re.sub(r"from [\w\.]+ import [\w\, ]+|import [\w\.]+", "", original_code, flags=re.MULTILINE)

    # 使用正则表达式提取需要导入的模块名称
    modules_to_import = ['Tools.tools']

    # 准备安全环境
    safe_globals = {
        "print": print,
        "__builtins__": {"print": print, "range": range},
    }
    
    def import_dependencies(dependencies, module):
        """
        导入函数的依赖。
        """
        for dep in dependencies:
            # 假设依赖的函数在同一个模块中
            if hasattr(module, dep):
                safe_globals[dep] = getattr(module, dep)
    # 动态导入模块并将函数添加到 safe_globals 中
    for function in function_manager.functions_info['functions']:
        file_path = function['file_path'].replace('\\', '.')  # 将文件路径转换为模块路径
        module_path = re.sub(r'\.py$', '', file_path)  # 移除.py扩展名
        module = importlib.import_module(module_path)  # 动态导入模块

        # 获取函数并添加到safe_globals
        func = getattr(module, function['name'])
        safe_globals[function['name']] = func
        
    # 导入函数的依赖
    if function['dependencies']:
        import_dependencies(function['dependencies'], module)
    # 执行代码
    try:
        exec(code_string, safe_globals)
        if 'result' in safe_globals and 'used_function_name' in safe_globals:
            execution_result = {"function_name": safe_globals['used_function_name'], "result": safe_globals['result']}
            print("Execution result:", execution_result)
        else:
            print("No result or used function name variable found in executed code.")
    except Exception as e:
        print("Error executing code:", e)

else:
    print("No code found.")

[系统]: 对话已命名为 '```python
from Tools.tools import sum
result = sum(3, 5)
used_function_name = 'sum'
```' 并保存。
Execution result: {'function_name': 'sum', 'result': 8}


In [ ]:
service_type='sensetime'

if service_type in ['sensetime', None]:
    service = AuthenticatedRequestSender()
elif service_type == 'openai':
    service = OpenAIService()
else:
    raise ValueError('未知的服务类型')
chat=JoiningChat(service_type, root_path)
localmemory=LocalMemory(service_type, root_path)
question='请问你玩原神吗'
chat.process_input(question)

In [2]:
project_root = root_path
function_manager = FunctionManager(project_root, service_type='sensetime')
function_manager.update_func_json()

In [3]:
print(function_manager.functions_info)

{'functions': [{'name': 'sum', 'description': "The function 'sum' takes two arguments, 'a' and 'b', and returns their sum.", 'input': ['a', 'b'], 'output': "BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))", 'file_path': 'Tools\\functions_info.json', 'dependencies': []}]}


In [13]:
pip install scipy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import os
import whisper

# 录音参数
fs = 44100  # 采样率
duration = 5  # 录音时间，秒
channels = 2  # 通道数

temp_sound_dir = os.path.join(root_path, "TempSound")
if not os.path.exists(temp_sound_dir):
    os.makedirs(temp_sound_dir)

# 录音文件路径
audio_file_path = os.path.join(temp_sound_dir, "temp_recording.wav")

# 预先加载Whisper模型
print("正在加载模型，请稍候...")
model = whisper.load_model("large")
print("模型加载完成。")

C:\Users\Jiaoy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


正在加载模型，请稍候...


100%|█████████████████████████████████████| 2.88G/2.88G [10:33<00:00, 4.88MiB/s]


模型加载完成。


In [7]:
def record_audio(file_path, fs=44100, channels=2):
    print("按回车开始录音，再次按回车结束录音。")
    input()  # 等待回车键开始
    
    # 准备录音
    recording = []
    def callback(indata, frames, time, status):
        # 将录音数据实时添加到列表中
        recording.append(indata.copy())

    # 开始录音
    print("正在录音...按回车键结束。")
    with sd.InputStream(callback=callback, samplerate=fs, channels=channels):
        input()  # 再次等待回车键结束录音
    
    print("录音结束。")

    # 将录音数据转换为NumPy数组，并保存为WAV文件
    recording_array = np.concatenate(recording, axis=0)
    wav.write(file_path, fs, np.int16(recording_array * 32767))

def transcribe_audio(model, file_path):
    # 使用预先加载的Whisper模型进行语音识别
    result = model.transcribe(file_path)
    print("识别结果：", result["text"])

# 实际执行录音和语音识别
record_audio(audio_file_path)
transcribe_audio(model, audio_file_path)


按回车开始录音，再次按回车结束录音。
正在录音...按回车键结束。
录音结束。
识别结果： 你好,再见


In [ ]:
import re
import importlib
import os
import json
from dotenv import load_dotenv
from LLM_API import AuthenticatedRequestSender
from LLM_API import OpenAIService
from FileProcess import FileProcessor
from Memory import Embedding, LocalMemory
from Chat import JoiningChat
from WebExecuter import WebSearch
from FileProcess import CodeAnalyser
from Tools import FunctionManager

def execute_function_from_description(question, function_manager=None, chat=None, fileprocessor=None):
    dotenv_path = os.path.join(os.getcwd(), '.env')
    root_path = dotenv_path.rstrip('.env')
    load_dotenv(dotenv_path)

    # 检查并初始化 FileProcessor
    if fileprocessor is None:
        fileprocessor = FileProcessor(root_path)

    # 检查并初始化 FunctionManager
    if function_manager is None:
        project_root = root_path
        service_type = 'sensetime'
        function_manager = FunctionManager(project_root, service_type=service_type)

    # 检查并初始化 JoiningChat
    if chat is None:
        service_type = 'sensetime'
        chat = JoiningChat(service_type=service_type, root_path=root_path)

    ask = f"Based on the following function descriptions: '{function_manager.functions_info}', could you demonstrate how to address '{question}'? Please include a specific code snippet that performs this operation, assigns the result to a variable named 'result', and also assigns the name of the used function to a variable named 'used_function_name'."
    answer = chat.process_input(ask)

    pattern = r"```python\n(.*?)\n```"
    match = re.search(pattern, answer, re.DOTALL)
    if match:
        original_code = match.group(1)
        code_string = re.sub(r"from [\w\.]+ import [\w\, ]+|import [\w\.]+", "", original_code, flags=re.MULTILINE)
        safe_globals = {
            "print": print,
            "__builtins__": {"print": print, "range": range},
        }

        for function in function_manager.functions_info['functions']:
            file_path = function['file_path'].replace('\\', '.')
            module_path = re.sub(r'\.py$', '', file_path)
            module = importlib.import_module(module_path)
            func = getattr(module, function['name'])
            safe_globals[function['name']] = func
            if function['dependencies']:
                for dep in function['dependencies']:
                    if hasattr(module, dep):
                        safe_globals[dep] = getattr(module, dep)

        try:
            exec(code_string, safe_globals)
            if 'result' in safe_globals and 'used_function_name' in safe_globals:
                execution_result = {"function_name": safe_globals['used_function_name'], "result": safe_globals['result']}
                return execution_result
            else:
                return "No result or used function name variable found in executed code."
        except Exception as e:
            return f"Error executing code: {e}"
    else:
        return "No code found."
